In [ ]:
from datetime import datetime
import bisect

# 좌석 상태 저장용 해시테이블
class HashTable:
    def __init__(self):
        self.table = {}

    def insert(self, key, value):
        self.table[key] = value

    def get(self, key):
        return self.table.get(key)

    def update(self, key, value):
        if key in self.table:
            self.table[key] = value

    def find_empty_seats(self):
        return [key for key, value in self.table.items() if value['status'] == 'notReserved']


# 시간순 정렬 예약 관리 클래스
class ReservationManager:
    def __init__(self, seat_table):
        self.sorted_reservations = []  # (time_slot, seat_id)
        self.seat_table = seat_table

    def reserve(self, seat_id, time_slot):
        bisect.insort(self.sorted_reservations, (time_slot, seat_id))
        self.seat_table.update(seat_id, {"status": "reserved", "time_slot": time_slot})

    def cancel(self, seat_id):
        time_slot = self.seat_table.get(seat_id)['time_slot']
        if (time_slot, seat_id) in self.sorted_reservations:
            self.sorted_reservations.remove((time_slot, seat_id))
        self.seat_table.update(seat_id, {"status": "notReserved", "time_slot": ""})

    def auto_cancel(self):
        current_time = datetime.now().strftime("%H:%M")
        to_cancel = []
        for time_slot, seat_id in self.sorted_reservations:
            if time_slot <= current_time:
                to_cancel.append((time_slot, seat_id))
            else:
                break
        for time_slot, seat_id in to_cancel:
            print(f"[자동 취소] 좌석 {seat_id}의 예약({time_slot})이 현재 시간({current_time}) 기준으로 취소되었습니다.")
            self.cancel(seat_id)

    def display(self):
        print("\n[시간 순 예약 현황]")
        if not self.sorted_reservations:
            print("현재 예약된 좌석이 없습니다.")
            return
        for time_slot, seat_id in self.sorted_reservations:
            print(f"Seat {seat_id} - Time Slot: {time_slot}")


# 예약 함수
def reserve_seat(seat_table, manager):
    empty_seats = seat_table.find_empty_seats()
    if not empty_seats:
        print("빈 좌석이 없습니다.")
        return

    print(f"예약 가능한 좌석: {', '.join(map(str, empty_seats))}")
    seat_id = input("예약할 좌석 번호 입력: ")
    if not seat_id.isdigit() or int(seat_id) not in empty_seats:
        print("잘못된 좌석 번호입니다.")
        return
    seat_id = int(seat_id)

    time_slot = input("예약할 시간대 입력 (예: 17:00): ")
    manager.reserve(seat_id, time_slot)
    print(f"{seat_id}번 좌석이 {time_slot}에 예약되었습니다.")


# 취소 함수
def cancel_reservation(seat_table, manager):
    seat_id = input("취소할 좌석 번호 입력: ")
    if not seat_id.isdigit():
        print("잘못된 입력입니다.")
        return
    seat_id = int(seat_id)

    seat_data = seat_table.get(seat_id)
    if seat_data is None or seat_data['status'] != 'reserved':
        print("해당 좌석은 예약되지 않았습니다.")
        return

    manager.cancel(seat_id)
    print(f"{seat_id}번 좌석 예약이 취소되었습니다.")


# 메인
def main():
    seat_table = HashTable()
    for i in range(1, 11):
        seat_table.insert(i, {"status": "notReserved", "time_slot": ""})

    manager = ReservationManager(seat_table)

    while True:
        print("\n=== 도서관 좌석 예약 시스템 ===")
        print("1. 예약")
        print("2. 취소")
        print("3. 자동 예약 취소 실행")
        print("4. 시간순 예약 현황 보기")
        print("5. 종료")
        choice = input("메뉴 선택: ")

        if choice == "1":
            reserve_seat(seat_table, manager)
        elif choice == "2":
            cancel_reservation(seat_table, manager)
        elif choice == "3":
            manager.auto_cancel()
        elif choice == "4":
            manager.display()
        elif choice == "5":
            print("시스템을 종료합니다.")
            break
        else:
            print("잘못된 입력입니다.")


if __name__ == "__main__":
    main()


=== 도서관 좌석 예약 시스템 ===
1. 예약
2. 취소
3. 자동 예약 취소
4. 종료

=== 도서관 좌석 예약 시스템 ===
1. 예약
2. 취소
3. 자동 예약 취소
4. 종료

=== 예약 ===
현재 빈 좌석 번호: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
좌석 5 예약이 완료되었습니다. 예약 시간대: 17:99

=== 도서관 좌석 예약 시스템 ===
1. 예약
2. 취소
3. 자동 예약 취소
4. 종료

=== 도서관 좌석 예약 시스템 ===
1. 예약
2. 취소
3. 자동 예약 취소
4. 종료

=== 예약 취소 ===
좌석 1는 예약되지 않은 좌석입니다. 취소할 수 없습니다.

=== 도서관 좌석 예약 시스템 ===
1. 예약
2. 취소
3. 자동 예약 취소
4. 종료
잘못된 입력입니다. 다시 선택해주세요.

=== 도서관 좌석 예약 시스템 ===
1. 예약
2. 취소
3. 자동 예약 취소
4. 종료
잘못된 입력입니다. 다시 선택해주세요.

=== 도서관 좌석 예약 시스템 ===
1. 예약
2. 취소
3. 자동 예약 취소
4. 종료
